<a href="https://colab.research.google.com/github/joeunyook/Genies/blob/main/htf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install google-generativeai python-dotenv
!pip install Pillow # Required for image handling
!pip install psycopg2-binary # For PostgreSQL interaction


In [48]:
import google.generativeai as genai
import os
from PIL import Image  # Import Pillow library
import random  # For mock damage classifier
import psycopg2  # Uncomment if you're using PostgreSQL
from dotenv import load_dotenv

# 3. Load API Key (from Google Drive - Secure Method)
from google.colab import drive
drive.mount('/content/drive')

try:
    with open("/content/drive/MyDrive/htf/gemini_api_key.txt", "r") as file:
        GEMINI_API_KEY = file.read().strip()
    print("API key loaded from Google Drive.")  # Added print statement
    genai.configure(api_key=GEMINI_API_KEY)  # Configure Gemini AI
except FileNotFoundError:
    print("Error: API key file not found in Google Drive.  Make sure the path is correct.")
    GEMINI_API_KEY = None # Handle the case where the key isn't loaded

except Exception as e:
    print(f"Error loading API key: {e}")
    GEMINI_API_KEY = None # Handle potential errors during key loading

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
API key loaded from Google Drive.


In [49]:
# 4 .Gemini API Classification Function
def classify_book_damage(image_path):
    """
    Uses Gemini API to classify book damage and returns type and severity.
    """
    if not GEMINI_API_KEY:
        print("Gemini API key not loaded.")
        return None

    try:
        model_name = "gemini-2.0-flash"  # Or correct model
        model = genai.GenerativeModel(model_name)

        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Error: Image not found: {image_path}")
            return None
        except Exception as e:
            print(f"Error reading image: {e}")
            return None

        damage_options = ["corner_damage", "cover_scratches", "spine_damage", "water_damage", "tears_or_rips", "misprints", "missing_dust_jacket", "trim_issues"] # Under score labels
        # **HIGHLY REFINED PROMPT (Underscore Labels):**
        prompt = f"""Carefully analyze the image of the book.
        Identify the *single* most significant type of damage present.
        You *must* choose one of the following damage types (use *exactly* these labels): {', '.join(damage_options)}.

        After identifying the damage type, assess its severity on a scale of 1 to 5, where:
        1: Very Minor Damage
        2: Minor Damage
        3: Moderate Damage
        4: Significant Damage
        5: Severe Damage

        Return the result in *exactly* the following format. Do not include any other text:
        Damage Type: [damage_type]
        Severity: [severity_level]

        Example:
        Damage Type: corner_damage
        Severity: 3
        """

        response = model.generate_content([prompt, image])

        print("Gemini API response received.")
        print(f"Gemini API response: {response.text}")
        return response.text

    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        return None


In [50]:
def extract_damage_info(gemini_response):
    """
    Extracts damage type and severity from the Gemini API response (Underscore Labels).
    """
    damage_match = re.search(r"Damage Type:\s*([a-z_]+)", gemini_response)  # Only lowercase letters and underscores
    severity_match = re.search(r"Severity:\s*(\d+)", gemini_response)

    if damage_match and severity_match:
        damage_type = damage_match.group(1).strip()  # No need to lowercase
        severity = int(severity_match.group(1))

        print(f"Extracted Damage Type: {damage_type}")
        print(f"Extracted Severity: {severity}")
        return {"type": damage_type, "severity": severity}
    else:
        print("Could not extract damage type and severity.")
        return None


In [51]:
# 6. Price Point Determiner
def calculate_discounted_price(original_price, damage_type, severity, devaluation_rate=0.0):
    """
    Calculates the discounted price based on publisher rules and dynamic devaluation.
    """
    try:
        discount_rate = publisher_rules[damage_type][severity]
        print(f"Base Discount Rate: {discount_rate}")  # Print base discount rate

        # Apply dynamic devaluation rate (e.g., 2% = 0.02)
        adjusted_discount_rate = discount_rate * (1 - devaluation_rate)
        print(f"Adjusted Discount Rate: {adjusted_discount_rate} (after {devaluation_rate*100}% devaluation)")  # Print adjusted rate

        discounted_price = original_price * (1 - adjusted_discount_rate)
        print(f"Calculated Discounted Price: ${discounted_price:.2f}")
        return discounted_price
    except KeyError:
        print(f"Error: No rule found for damage type '{damage_type}' severity '{severity}'.")
        return None


In [44]:

'''
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

# Define request model
class TradeRequest(BaseModel):
    discounted_price: float
    grace_range_low: float
    grace_range_high: float
    auto_trade_enabled: bool

# 7. Auto Trader with Manual Toggle from Frontend
def auto_trade_decision(discounted_price, grace_range_low, grace_range_high, auto_trade_enabled):
    """
    Determines trade action based on user selection from frontend.

    If auto_trade_enabled is True:
        - Automatically purchases the book if it's within the grace range.
        - Otherwise, puts it up for auction.

    If auto_trade_enabled is False:
        - Always puts the book up for auction.
    """
    if auto_trade_enabled:
        if grace_range_low <= discounted_price <= grace_range_high:
            return {"trade_decision": "automatic purchase"}
        else:
            return {"trade_decision": "auction based"}
    else:
        return {"trade_decision": "auction based"}

# FastAPI route to handle trade decision request
@app.post("/trade-decision")
def trade_decision(request: TradeRequest):
    decision = auto_trade_decision(
        request.discounted_price,
        request.grace_range_low,
        request.grace_range_high,
        request.auto_trade_enabled
    )
    return decision
'''

'\nfrom fastapi import FastAPI\nfrom pydantic import BaseModel\n\napp = FastAPI()\n\n# Define request model\nclass TradeRequest(BaseModel):\n    discounted_price: float\n    grace_range_low: float\n    grace_range_high: float\n    auto_trade_enabled: bool\n\n# 7. Auto Trader with Manual Toggle from Frontend\ndef auto_trade_decision(discounted_price, grace_range_low, grace_range_high, auto_trade_enabled):\n    """\n    Determines trade action based on user selection from frontend.\n    \n    If auto_trade_enabled is True:\n        - Automatically purchases the book if it\'s within the grace range.\n        - Otherwise, puts it up for auction.\n        \n    If auto_trade_enabled is False:\n        - Always puts the book up for auction.\n    """\n    if auto_trade_enabled:\n        if grace_range_low <= discounted_price <= grace_range_high:\n            return {"trade_decision": "automatic purchase"}\n        else:\n            return {"trade_decision": "auction based"}\n    else:\n     

In [54]:
# 8. Main Function
def process_book_return(image_path, original_price, devaluation_rate=0.0):
    """
    Main function to orchestrate the process.  Now with dynamic devaluation.
    """
    damage_classification = classify_book_damage(image_path)

    if damage_classification:
        damage_info = extract_damage_info(damage_classification)

        if damage_info:
            # Call calculate_discounted_price with the devaluation rate
            discounted_price = calculate_discounted_price(original_price, damage_info["type"], damage_info["severity"], devaluation_rate)

            if discounted_price is not None:
                print(f"Damage Type: {damage_info['type']}")
                print(f"Severity: {damage_info['severity']}")
                print(f"Original Price: ${original_price:.2f}")
                print(f"Final Discounted Price: ${discounted_price:.2f}")
            else:
                print("Could not determine discounted price due to missing rule.")
        else:
            print("Could not extract damage information.")
    else:
        print("Could not classify damage from the image.")

# Example Usage (Inside the if __name__ == "__main__": block)


# Added Publisher Rules
publisher_rules = {
    "corner_damage": { #Updated keys to underscore format
        1: 0.05,
        2: 0.10,
        3: 0.22,
        4: 0.50,
        5: 0.75
    },
    "cover_scratches": { #Updated keys to underscore format
        1: 0.02,
        2: 0.05,
        3: 0.15,
        4: 0.30,
        5: 0.50
    },
    "spine_damage": { #Updated keys to underscore format
        1: 0.10,
        2: 0.20,
        3: 0.40,
        4: 0.65,
        5: 0.85
    },
    "water_damage": { #Updated keys to underscore format
        1: 0.20,
        2: 0.40,
        3: 0.60,
        4: 0.80,
        5: 0.95
    },
    "tears_or_rips": { #Updated keys to underscore format
        1: 0.05,
        2: 0.10,
        3: 0.30,
        4: 0.55,
        5: 0.75
    },
    "misprints": { #Updated keys to underscore format
        1: 0.05,
        2: 0.15,
        3: 0.30,
        4: 0.50,
        5: 0.80
    },
    "missing_dust_jacket": { #Updated keys to underscore format
        1: 0.05,
        2: 0.10,
        3: 0.20,
        4: 0.35,
        5: 0.50
    },
    "trim_issues": { #Updated keys to underscore format
        1: 0.02,
        2: 0.07,
        3: 0.20,
        4: 0.40,
        5: 0.60
    }
}



In [55]:
# 9. Example Usage
if __name__ == "__main__":
    # **UPLOAD AN IMAGE TO GOOGLE COLAB FIRST!**
    image_path = "/content/drive/MyDrive/htf/book_example.jpg"  # Replace with the actual image path
    original_price = 20.00

    process_book_return(image_path, original_price)


Gemini API response received.
Gemini API response: Damage Type: corner_damage
Severity: 3
Extracted Damage Type: corner_damage
Extracted Severity: 3
Base Discount Rate: 0.22
Adjusted Discount Rate: 0.22 (after 0.0% devaluation)
Calculated Discounted Price: $15.60
Damage Type: corner_damage
Severity: 3
Original Price: $20.00
Final Discounted Price: $15.60
